In [15]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
from IPython import embed
from collections import OrderedDict
import numpy as np

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

# Torchvision
import torchvision
import torchvision.transforms as transforms

# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# OS
import os
import argparse

model_urls = {
    'cifar10': 'http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/cifar10-d875770b.pth',
    'cifar100': 'http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/cifar100-3a55a987.pth',
}

class CIFAR(nn.Module):
    def __init__(self, features, n_channel, num_classes):
        super(CIFAR, self).__init__()
        assert isinstance(features, nn.Sequential), type(features)
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(n_channel, num_classes)
        )
        # print(self.features)
        # print(self.classifier)
    def embedding(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        # x = self.classifier(x)
        return x

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for i, v in enumerate(cfg):
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            padding = v[1] if isinstance(v, tuple) else 1
            out_channels = v[0] if isinstance(v, tuple) else v
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=padding)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(out_channels, affine=False), nn.ReLU()]
            else:
                layers += [conv2d, nn.ReLU()]
            in_channels = out_channels
    return nn.Sequential(*layers)

def cifar10(n_channel, pretrained=None):
    cfg = [n_channel, n_channel, 'M', 2*n_channel, 2*n_channel, 'M', 4*n_channel, 4*n_channel, 'M', (8*n_channel, 0), 'M']
    layers = make_layers(cfg, batch_norm=True)
    model = CIFAR(layers, n_channel=8*n_channel, num_classes=10)
    if pretrained is not None:
        m = model_zoo.load_url(model_urls['cifar10'])
        state_dict = m.state_dict() if isinstance(m, nn.Module) else m
        assert isinstance(state_dict, (dict, OrderedDict)), type(state_dict)
        model.load_state_dict(state_dict)
    return model

def cifar100(n_channel, pretrained=None):
    cfg = [n_channel, n_channel, 'M', 2*n_channel, 2*n_channel, 'M', 4*n_channel, 4*n_channel, 'M', (8*n_channel, 0), 'M']
    layers = make_layers(cfg, batch_norm=True)
    model = CIFAR(layers, n_channel=8*n_channel, num_classes=100)
    if pretrained is not None:
        m = model_zoo.load_url(model_urls['cifar100'])
        state_dict = m.state_dict() if isinstance(m, nn.Module) else m
        assert isinstance(state_dict, (dict, OrderedDict)), type(state_dict)
        model.load_state_dict(state_dict)
    return model

In [16]:
# gan_model = cifar10(128, pretrained='log/cifar10/best-135.pth')

In [17]:
# torch.save(img_syn, 'img_syn.pt')
# torch.save(label_syn, 'label_syn.pt')
# # 读取tensor
img_syn = torch.load('img_syn.pt')
label_syn = torch.load('label_syn.pt')

device  = "cuda:0"
img_syn =img_syn.to(device)
label_syn =label_syn.to(device)
device = img_syn.device

# Create model
gan_model = cifar10(128).to(device)


In [26]:

parser = argparse.ArgumentParser(description='Parameter Processing')
parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
parser.add_argument('--model', type=str, default='ConvNet', help='model')
parser.add_argument('--ipc', type=int, default=50, help='image(s) per class')
parser.add_argument('--eval_mode', type=str, default='SS', help='eval_mode') # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
parser.add_argument('--num_exp', type=int, default=1, help='the number of experiments')
parser.add_argument('--num_eval', type=int, default=1, help='the number of evaluating randomly initialized models')
parser.add_argument('--epoch_eval_train', type=int, default=1000, help='epochs to train a model with synthetic data') # it can be small for speeding up with little performance drop
parser.add_argument('--Iteration', type=int, default=2000, help='training iterations')
parser.add_argument('--lr_img', type=float, default=1.0, help='learning rate for updating synthetic images')
parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
parser.add_argument('--batch_real', type=int, default=256, help='batch size for real data')
parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
parser.add_argument('--init', type=str, default='real', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
parser.add_argument('--dsa_strategy', type=str, default='color_crop_cutout_flip_scale_rotate', help='differentiable Siamese augmentation strategy')
parser.add_argument('--data_path', type=str, default='/home/ssd7T/ZTL_gcond/data_cv', help='dataset path')
parser.add_argument('--save_path', type=str, default='result/gen', help='path to save results')
parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric')
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import warnings
args = parser.parse_args([])
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
labels_all = [dst_train[i][1] for i in range(len(dst_train))]
indices_class = [[] for c in range(num_classes)]
for i, lab in enumerate(labels_all):
    indices_class[lab].append(i)
images_all = torch.cat(images_all, dim=0).to(device)
labels_all = torch.tensor(labels_all, dtype=torch.long, device=device)

accs = []
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
args.device = "cuda:0"
import copy
accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
model_eval= model_eval_pool[0]
data_save = []

# img_real = []
# label_real = []
# for c in range(num_classes):
#     idx_shuffle = np.random.permutation(indices_class[c])
#     img_real.append(images_all[idx_shuffle].to("cpu") )
#     label_real.append(labels_all[idx_shuffle].to("cpu"))
# img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
# label_real = torch.from_numpy(np.concatenate(label_real, axis=0))


SEED = 114514
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    
img_real_train = []
label_real_train = []
for c in range(num_classes):
    idx_shuffle = np.random.permutation(indices_class[c])
    img_real_train.append(images_all[idx_shuffle].to("cpu") )
    label_real_train.append(labels_all[idx_shuffle].to("cpu"))
img_real_train = torch.from_numpy(np.concatenate(img_real_train, axis=0))
label_real_train = torch.from_numpy(np.concatenate(label_real_train, axis=0))

SEED = 87
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
img_real_test = []
label_real_test = []
for c in range(num_classes):
    idx_shuffle = np.random.permutation(indices_class[c])[:50]
    img_real_test.append(images_all[idx_shuffle].to("cpu") )
    label_real_test.append(labels_all[idx_shuffle].to("cpu"))
img_real_test = torch.from_numpy(np.concatenate(img_real_test, axis=0))
label_real_test = torch.from_numpy(np.concatenate(label_real_test, axis=0))

Files already downloaded and verified
Files already downloaded and verified


In [19]:
import argparse
import os
import time

# from utee import misc
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from IPython import embed

parser = argparse.ArgumentParser(description='PyTorch CIFAR-X Example')
parser.add_argument('--type', default='cifar10', help='cifar10|cifar100')
parser.add_argument('--channel', type=int, default=128, help='first conv channel (default: 32)')
parser.add_argument('--wd', type=float, default=0.00, help='weight decay')
parser.add_argument('--batch_size', type=int, default=50, help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=150, help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate (default: 1e-3)')

parser.add_argument('--log_interval', type=int, default=100,  help='how many batches to wait before logging training status')
parser.add_argument('--test_interval', type=int, default=5,  help='how many epochs to wait before another test')

parser.add_argument('--decreasing_lr', default='80,120', help='decreasing strategy')
args = parser.parse_args([])
SEED = 87
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


In [20]:


num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.BCELoss()
# model = Autoencoder(num_feat).to(device)  
# 训练

# optimizer
optimizer = optim.Adam(gan_model.parameters(), lr=args.lr, weight_decay=args.wd)
decreasing_lr = list(map(int, args.decreasing_lr.split(',')))
print('decreasing_lr: ' + str(decreasing_lr))


best_acc, old_file = 0, None
t_begin = time.time()

# ready to go
for epoch in range(2000):
  
    gan_model.train()
    if epoch in decreasing_lr:
        optimizer.param_groups[0]['lr'] *= 0.1
    for c in range(num_classes):
        batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
        # label_real_train = label_real_train.to(device) 
        batch_label = label_syn[c*batch:(c+1)*batch].to(device) 
        # batch_img = batch_img.to(device) 

        optimizer.zero_grad()
        output = gan_model(batch_img)
        loss = F.cross_entropy(output, batch_label)
        loss.backward()
        optimizer.step()

    

        

decreasing_lr: [80, 120]


In [21]:
# with torch.no_grad():
#   output = autoencoder(img_real_test.to(device))
  
  
  
gan_model.eval()
test_loss = 0
correct = 0

output = gan_model.embedding(img_real_test.to(device))
    # test_loss += F.cross_entropy(output, target).data[0]
    # pred = output.data.max(1)[1]  # get the index of the max log-probability
    # correct += pred.cpu().eq(indx_target).sum()

    #         test_loss = test_loss / len(test_loader) # average over number of mini-batch
    #         acc = 100. * correct / len(test_loader.dataset)
    #         print('\tTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
    #             test_loss, correct, len(test_loader.dataset), acc))
    #         if acc > best_acc:
    #             new_file = os.path.join(args.logdir, 'best-{}.pth'.format(epoch))
    #             misc.model_snapshot(model, new_file, old_file=old_file, verbose=True)
    #             best_acc = acc
    #             old_file = new_file

  

In [32]:
output = gan_model(img_real_test.to(device))

In [33]:
pred = output.data.max(1)[1]  # get the index of the max log-probability

In [34]:
correct += pred.cpu().eq(label_real_test).sum()

In [36]:
acc = 100. * correct / len(output)
print('Test set: Accuracy: {:.0f}%'.format(acc))

Test set: Accuracy: 9%


In [27]:

accs = []
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
args.device = "cuda:0"
import copy
accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
model_eval= model_eval_pool[0]

# [2023-10-06 20:45:24] Evaluate_00: epoch = 1000 train time = 61 s train loss = 0.000965 train acc = 1.0000, test acc = 0.5001
# Evaluate 1 random ConvNet, mean = 0.5001 std = 0.0000

In [31]:

net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output.detach()), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)